## Wykonywanie równoległe
- Architektura master - worker.
- master - proces, z którego sterujemy wykonaniem programu
- master zleca wykonanie danej funkcji na innym procesie i ściąga wynik

In [ ]:
# Uruchomienie procesów workerów (zwykle podajemy tyle, ile węzłów chcemy wykorzystać)
# jesli uruchamiamy konsole julii to można też użyć opcji 
# julia -p ilosc_workerów
# do usuwania workerów służy  rmprocs() - tylko dla procesu master !

function set_workers(target)
    if nprocs() == 1
        addprocs(1)
    end
        
    current = nworkers()
    if current < target
        addprocs(target - current)
    elseif current > target
        rmprocs(workers()[end - (current - target - 1):end])
    end
end

set_workers(4)
workers()

### Zbiór Julii

### Zadanie

- Mając dany program obliczający i rysujący zbiór Julii, napisać jego wersję równoległą w dwóch  wariantach: (1) wykorzystując parallel for, (2) wykorzystując pmap, 

- Proszę zmierzyć przyspieszenie i efektywność dla zmiennej liczby workerów (od 1 do liczby rdzeni w procesorze). Wyniki porównania proszę przedstawić na wykresie. 

Uwagi:

- Pomocny może być przykład pokazujący <a href="http://distrustsimplicity.net/articles/mandelbrot-speed-comparison/">równoległą implementację zbioru Mandelbrota</a> w oparciu o parallel for.

- Pojedyncze zadanie powinno obliczać jedną lub (najlepiej) więcej kolumn. Jeśli zadania będą zbyt drobne, to koszt wynikający z narzutu zrównoleglania (przełączanie między zadaniami, komunikacja między procesami) przewyższy zyski z podziału pracy na wiele procesów.


In [ ]:
using Plots
Plots.gr()

# punkt o współrzędnych (x,y) należy do zbioru Julii o parametrze  c 
# jeśli dla liczby zespolonej z=x+i*y
# ciąg zₙ₊₁=zₙ²+c , nie dąży do nieskończoności

# dążenie do nieskończoności sprawdzamy ustawiająć maksymalną liczbę iteracji i sprawdzając
# czy kolejne wyrazy ciągu nie przekroczą zadanego progu (tutaj 2) w tej (bądź mniejszej)
# liczbie iteracji

# funkcja sprawdzająca, czy punkt z należy do zbioru Julii o parametrze c
@everywhere function generate_julia(z; c=2, maxiter=200)
    for i=1:maxiter
        if abs(z) > 2
            return i-1
        end
        z = z^2 + c
    end
    return maxiter
end

# wypełnianie tablicy liczbami złożonymi
function initialize_array(array, height, width, xrange, yrange)
    for x in 1:width
        for y in 1:height
            array[x, y] = xrange[x] + 1im * yrange[y]
        end
    end
    return array
end

# główna funkcja 
function main(height, width, julia_calculator, array_factory, make_plot=true)
    # ustawiamy płaszczyznę
    xmin, xmax = -2,2
    ymin, ymax = -1,1
    xrange = linspace(xmin, xmax, width)
    yrange = linspace(ymin, ymax, height)
    
    # obliczamy
    julia_set = array_factory(width, height)
    initialize_array(julia_set, height, width, xrange, yrange)
    println(@elapsed(julia_set = julia_calculator(julia_set, height, width)))
    
    # rysujemy
    if make_plot
        return Plots.heatmap(xrange, yrange, convert(Array{Int64}, julia_set))
    end
end

In [ ]:
function standard_array(width, height)
    return Array{Complex{Float64}}(width, height)
end

function shared_array(width, height)
    return SharedArray{Complex{Float64}}(width, height)
end

function sequential_calculator(array, height, width, maxiter=200)
   for x in 1:height
        for y in 1:width
            array[x, y] = generate_julia(array[x, y], c=-0.70176-0.3842im, maxiter=maxiter)
        end
    end
    return array
end

function parallel_calculator(array, height, width, maxiter=200)    
    @sync @parallel for x in 1:height
        for y in 1:width
            array[x, y] = generate_julia(array[x, y], c=-0.70176-0.3842im, maxiter=maxiter)
        end
    end
    return array
end

function pmap_calculator(array, height, width, maxiter=200)
    pmap_fun(z) = convert(Complex{Float64}, generate_julia(z, c=-0.70176-0.3842im, maxiter=maxiter))
    return pmap(pmap_fun, array)
end

In [ ]:
repeats = 8

for workers in 4:-1:1
    set_workers(workers)
    for param in [("paralel", parallel_calculator, shared_array), ("pmap", pmap_calculator, shared_array)]
        (name, julia_calculator, array_factory) = param
        for n in [500, 2000, 5000]
            for i in 1:repeats
                print(name, "\t", nworkers(), "\t", n, "\t", i, "\t")
                main(n, n, julia_calculator, array_factory, false)
            end
        end
    end
end

for n in [500, 2000, 5000]
    for i in 1:repeats
        print("seq", "\t", nworkers(), "\t", n, "\t", i, "\t")
        main(n, n, sequential_calculator, standard_array, false)
    end
end

# main(n, n, sequential_calculator, standard_array)
# main(n, n, parallel_calculator, shared_array)
# main(n, n, pmap_calculator, shared_array)